In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer, SFTConfig
import bitsandbytes as bnb
import torch
import os

c:\ProgramData\Anaconda3\envs\llm_final\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

model_pretrained_name =  "./models/EXAONE-3.5-7.8B-Instruct/"
path_to_lora_adapters = "./models/EXAONE-3.5-7.8B-Instruct-SFT/checkpoint-32"

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_dtype="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
    
)
model = AutoModelForCausalLM.from_pretrained(
    model_pretrained_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True)
model = PeftModel.from_pretrained(model, path_to_lora_adapters)


Unused kwargs: ['bnb_4bit_quant_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████| 7/7 [00:32<00:00,  4.68s/it]


In [4]:
# new_model_name = './TuningModel/checkpoint-100/'

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer=AutoTokenizer.from_pretrained(model_pretrained_name)

In [6]:
# model = AutoModelForCausalLM.from_pretrained(new_model_name, load_in_4bit=True, device_map="auto", trust_remote_code=True).eval()

In [1]:
######중요######
###GPU 초기화###



import gc
import torch
gc.collect()



torch.cuda.empty_cache()

In [ ]:
prompt = "입영일자 연기 제도란 무엇인가요?"

messages = [{"role" : "system", "content" : "You are EXAONE model from LG AI Research, a helpful assistant."}, {"role" : "user", "content" : prompt}]

input = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
)

output = model.generate(
    input.to("cuda"),
    eos_token_id = tokenizer.eos_token_id,
    max_new_tokens = 1024,
    do_sample = False,
)

print(tokenizer.decode(output[0]))

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_pretrained_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model = PeftModel.from_pretrained(
    model,
    path_to_lora_adapters,
    torch_dtype = torch.float16
)

In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
)

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_id = './models/EXAONE-3.5-7.8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = bnb_config, 
                                             device_map="auto",
                                             trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 7/7 [00:05<00:00,  1.32it/s]


In [1]:
tokenizer.eos_token_id

NameError: name 'tokenizer' is not defined

In [4]:
prompt = "입영일자 연기 신청은 언제까지 가능한가요?"

messages = [{"role" : "system", "content" : "You are EXAONE model from LG AI Research, a helpful assistant."}, {"role" : "user", "content" : prompt}]

input = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
)

output = model.generate(
    input.to("cuda"),
    eos_token_id = tokenizer.eos_token_id,
    max_new_tokens = 256,
    do_sample = False,
)

print(tokenizer.decode(output[0]))

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]
[|user|]입영일자 연기 신청은 언제까지 가능한가요?
[|assistant|]입영일자 연기 신청 기간은 정확히 정해져 있지 않고, **개별 상황에 따라 달라집니다**. 

일반적으로 다음과 같은 요소들이 연기 신청 가능 기간에 영향을 미칩니다.

* **입영 예정일**:  입영일이 다가올수록 연기 신청이 제한될 가능성이 높습니다.
* **연기 사유의 타당성**:  병역청이 인정하는 합당한 사유가 있어야 연기가 승인될 가능성이 높습니다.
* **병역청의 정책**:  시기별로 병역청의 연기 신청 정책이 변경될 수 있습니다.

**정확한 정보는** **병역청 홈페이지**나 **직접 문의**를 통해 확인하시는 것이 가장 좋습니다. 

LG AI 연구원에서 개발된 EXAONE은 정확한 법률 및 행정 정보 제공에는 한계가 있으므로, 공식적인 출처를 참고하시길 권장합니다.[|endofturn|]


In [ ]:
prompt = "입영일자 연기 신청은 언제까지 가능한가요?"

messages = [{"role" : "system", "content" : "You are EXAONE model from LG AI Research, a helpful assistant."}, {"role" : "user", "content" : prompt}]

input = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
)

output = model.generate(
    input.to("cuda"),
    eos_token_id = tokenizer.eos_token_id,
    max_new_tokens = 256,
    do_sample = False,
)

print(tokenizer.decode(output[0]))

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.[|endofturn|]
[|user|]입영일자 연기 신청은 언제까지 가능한가요?
[|assistant|]현역병 입영 대상자로서 입영일자 연기를 희망하는 경우, 입영일자 연기신청서를 제출해야 하며, 연기신청은 입영일자 기준 3개월 전까지 가능합니다. 단, 병역이행일 연기사유가 있는 경우, 연기신청은 연기사유 발생일로부터 1년 이내에 제출해야 하며, 연기신청은 연기사유 기준 3개월 전까지 가능합니다. 신청인은 연기사유를 증명할 수 있는 서류를 제출해야 하며, 담당자 확인사항은 없습니다. 신청인은 담당자의 확인사항이 없으므로, 별도의 확인 절차 없이 제출하면 됩니다. 만약 연기신청서를 제출하지 않은 경우, 담당자는 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를 확인할 수 없습니다. 연기 여부는 담당자가 확인사항이 없으므로, 신청인은 연기 여부를